In [ ]:
import os
import numpy as np
import pandas as pd
import geopandas as gpd
from tqdm import tqdm
import hashlib

import dask
import xarray as xr
import xrspatial
from dask.diagnostics import ProgressBar
from geocube.api.core import make_geocube

import matplotlib.pyplot as plt
import seaborn as sns

PATH = "D:\World Bank\CLIENT v2"
DATA_RAW = rf"{PATH}\Data\Data_raw"
DATA_PROC = rf"{PATH}\Data\Data_proc"
DATA_OUT = rf"{PATH}\Data\Data_out"
GPW_PATH = rf"D:\Datasets\Gridded Population of the World"

In [ ]:
# floods = pd.read_csv(rf"{DATA_RAW}\Floods\GloFAS_floods.csv")

def load_population_data(bounds=None, generate=False):
    print("Processing Population data...")

    # Select all files in GPW folder
    files = os.listdir(GPW_PATH)
    files = [f for f in files if f.endswith(".tif")]
    
    # Compile into a single dataset
    dss = []
    for f in tqdm(files):
        
        ds = xr.open_dataset(os.path.join(GPW_PATH, f), chunks={"x": 1000, "y": 1000})
        ds["band_data"] = ds["band_data"].astype(np.uint32)
        if bounds is not None:
            ds = ds.sel(
                x=slice(bounds[0], bounds[2]), y=slice(bounds[3], bounds[1])
            )
        if generate:
            with ProgressBar():
                ds.sel(band=1).drop_vars("band").band_data.rio.to_raster(rf"{DATA_PROC}\{f.replace('.tif','_proc.tif')}")
                print(f"Saved {f.replace('.tif','_proc.tif')}")
        
        ds["year"] = int(f.split("_")[5])
        ds = ds.set_coords('year')
        dss += [ds]
        
    population = xr.concat(dss, dim="year")    
    
    # Filter if bounds are provided
    if bounds is not None:
        population = population.sel(
            x=slice(bounds[0], bounds[2]), y=slice(bounds[3], bounds[1])
        )
        
    # Clean band dimension
    population = population.sel(band=1).drop_vars(["band"])
    
    print("Done!")
    return population

def load_precipitation_data():
    era5 = xr.open_dataset(
        rf"{DATA_OUT}\ERA5_monthly_1970-2021_SPI-SPEI.nc",
        chunks={"latitude": 100, "longitude": 100},
    )
    era5 = era5.rename({"latitude": "y", "longitude": "x"})
    return

def load_IPUMS_country_data(wb_map):
    from osgeo import gdal
    import geopandas as gpd
    from IPython.display import display
    gdal.SetConfigOption('SHAPE_RESTORE_SHX', 'YES')

    print("Loading IPUMS country data...")

    path = rf"{DATA_RAW}\IPUMS Fixed"
    files = os.listdir(path)
    shpfiles = [f for f in files if ".shp" in f]
    geo2_files = [f for f in shpfiles if "geo2_" in f]
    geo2_countries = [name.split("_")[1][:2] for name in geo2_files]
    geo1_files = [f for f in shpfiles if "geo1_" in f]
    geo1_files = [f for f in geo1_files if f.split("_")[1][:2] not in geo2_countries]
    # return geo1_files, geo2_files
    files = geo2_files + geo1_files

    gdfs = []
    for f in tqdm(files):
        gdf = gpd.read_file(os.path.join(path, f))
        ctry_name  = gdf.CNTRY_NAME[0]
        
        # MANUAL FIXES
        if ctry_name == "United States":
            print("Removing Puerto Rico from US...")
            gdf = gdf[~gdf.GEOLEVEL2.isin(["840721079", "840721080", "840721081", "840721082", "840721083", "840721084", "840721085"])]
        if ctry_name == "Nigeria":
            print("Removing glitch from Nigeria...")
            gdf = gdf[gdf.GEOLEVEL2 != "566024008"]
        if ctry_name == "Israel":
            print("Normalizing Israel map to match WB...")
            # FIXME: there should be a better way to write this...
            fix = pd.read_csv(rf"{DATA_PROC}\fixes\fix_israel_geo2_adm1.csv", dtype={"GEOLEVEL2": str, "ADM1_CODE": int})
            gdf = gdf.merge(fix, on="GEOLEVEL2").drop(columns="geometry").drop_duplicates(subset="ADM1_CODE")
            gdf = wb_map[["geometry", "ADM1_CODE"]].merge(gdf, on="ADM1_CODE")
            gdf["GEOLEVEL2"] = gdf["ADM1_CODE"]
            gdf = gdf.drop(columns="ADM1_CODE")
        if ctry_name == "Palestine":
            print("Normalizing Palestine map to match WB...")
            fix = pd.read_csv(rf"{DATA_PROC}\fixes\fix_palestine_geo1_adm1.csv", dtype={"GEOLEVEL1": str, "ADM1_CODE": int})
            gdf = gdf.merge(fix, on="GEOLEVEL1").drop(columns="geometry").drop_duplicates(subset="ADM1_CODE")
            gdf = wb_map[["geometry", "ADM1_CODE"]].merge(gdf, on="ADM1_CODE")
            gdf["GEOLEVEL1"] = gdf["ADM1_CODE"]
            gdf = gdf.drop(columns="ADM1_CODE")

        gdfs += [gdf]
    
    gdf = gpd.GeoDataFrame(pd.concat(gdfs))
    gdf = gdf.set_crs("EPSG:4326")
    
    # Keep only relevant columns
    gdf.loc[gdf["GEOLEVEL1"].isna(), "GEOLEVEL1"] = gdf.loc[gdf["GEOLEVEL1"].isna(), "GEOLEVEL2"].str[:6] 
    id_cols = ["CNTRY_CODE", "GEOLEVEL1", "GEOLEVEL2"] 
    gdf = gdf[["geometry"] + id_cols]
    
    for col in id_cols:
        gdf[col] = pd.to_numeric(gdf[col]).fillna(0)
    gdf = gdf[gdf.GEOLEVEL1 != 0] # Drop unavailable data
    gdf = gdf[gdf.GEOLEVEL2 != 888888888] # Drop unavailable data
    assert gdf.duplicated(subset=id_cols).sum() == 0, "There are duplicated rows in the data!"
    
    # Create ID
    gdf["ID"] = gdf.groupby(id_cols).ngroup()
    assert gdf.ID.nunique() == gdf.shape[0], "ID is not unique!, there's some bug in the code..."
    print("Data loaded!")
    return gdf

def load_WB_country_data():
    print("Loading World Bank country data...")
    WB_country = gpd.read_file(rf"{DATA_RAW}\world_bank_adm2\world_bank_adm2.shp")
    
    # Assign nan when ADM2 is not available 
    WB_country.loc[WB_country.ADM2_NAME == "Administrative unit not available", "ADM2_CODE"] = (
        np.nan
    )
    
    # Create ADM_LAST variable: ADM2_NAME if available, else ADM1_NAME
    for col in ["ADM0_CODE", "ADM1_CODE", "ADM2_CODE"]:
        WB_country[col] = WB_country[col].fillna(0)

    # Dissolve by ADM_LAST and country code    
    WB_country = WB_country.dissolve(by=["ADM2_CODE","ADM1_CODE", "ADM0_CODE"]).reset_index()
    
    # # Create ID
    WB_country["ID"] = WB_country.groupby(["ADM2_CODE", "ADM1_CODE", "ADM0_CODE"]).ngroup()
    assert WB_country.ID.nunique() == WB_country.shape[0], "ID is not unique!, there's some bug in the code..."
    print("Data loaded!")
    return WB_country



def rasterize_shape_like_dataset(shape, dataset):
    print("Rasterizing shape...")
    raster = make_geocube(
        vector_data=shape,
        like=dataset,
    )
    # For some reason, like option is not working, so I have to manually add x and y
    assert (raster["x"].shape == dataset["x"].shape)
    assert (raster["y"].shape == dataset["y"].shape)
    raster["x"] = dataset["x"]
    raster["y"] = dataset["y"]
    raster = raster.drop_vars(["spatial_ref"])
    print("Done!")
    return raster

# Procesa IPUMS shapes

# Procesa WB/IPUMS shapes (administrative boundaries) y GPW (population data)

In [ ]:
WB_country = load_WB_country_data()
IPUMS_country = load_IPUMS_country_data(WB_country)
IPUMS_country = IPUMS_country.clip(WB_country.total_bounds)
population = load_population_data(bounds=WB_country.total_bounds)

In [ ]:
### Rasterize WB_country
WB_country_grid = rasterize_shape_like_dataset(
    WB_country[["ID", "geometry"]], 
    population
)

print("Saving WB_country_grid...")P
WB_country_grid.to_netcdf(rf"{DATA_PROC}\WB_country_grid.nc")
        
WB_country[["ID", "OBJECTID", "ADM2_CODE", "ADM1_CODE", "ADM0_CODE",  "geometry"]].to_feather(rf"{DATA_PROC}\WB_country_IDs.feather")


### Rasterize IPUMS_country
IPUMS_country_grid = rasterize_shape_like_dataset(
    IPUMS_country[["ID", "geometry"]], 
    population
)

IPUMS_country_path = rf"{DATA_PROC}\IPUMS_country_grid.nc"
print("Saving IPUMS_country_grid...")
IPUMS_country_grid.to_netcdf(rf"{DATA_PROC}\IPUMS_country_grid.nc")
        
IPUMS_country.to_feather(rf"{DATA_PROC}\IPUMS_country_IDs.feather")

# GENERA Base de droughts, con resolución original

In [ ]:
import utils
droughts_path = rf"{DATA_OUT}\ERA5_droughts_yearly.nc"

In [ ]:
# if not os.path.exists(droughts_path):
print("Preparing droughts dataset...")
# Genera base de sequías
era5 = xr.open_dataset(rf"{DATA_OUT}\ERA5_monthly_1970-2021_SPI-SPEI.nc", chunks={'latitude': 1000, 'longitude': 1000})
# Corrije la dimensión x, que va de 0 a 360
era5 = era5.rename({'latitude': 'y', 'longitude': 'x'})
era5 = utils.coordinates_from_0_360_to_180_180(era5) # FIXME: no se si esto está andando bien, pero creo que si. VERIFICAR

# Calcula las sequías anuales
spi_yearly = era5.groupby("time.year").min()
with ProgressBar():
    spi_yearly.to_netcdf(rf"{DATA_PROC}\ERA5_yearly_1970-2021_SPI-SPEI.nc")

In [ ]:
spi_yearly = xr.open_dataset(rf"{DATA_PROC}\ERA5_yearly_1970-2021_SPI-SPEI.nc", chunks={"x": 900, "y": 1800})

spi_spei_vars = [var for var in spi_yearly.data_vars if "-" in var]
for var in spi_spei_vars:
    for threshold_str in ["1_0", "1_5", "2_0", "2_5"]:
        threshold = float(threshold_str.replace("_", "."))
        threshold_str = threshold_str.replace("_", "")
        spi_yearly[f"drought_{var}_{threshold_str}sd"] = (spi_yearly[var] < -threshold).astype("bool")

spi_yearly = spi_yearly[[var for var in spi_yearly.data_vars if "drought" in var]]
spi_yearly = spi_yearly.rename({
    var: var.replace("drought_", "").replace("-", "") for var in spi_yearly.data_vars
})
with ProgressBar():
    spi_yearly.to_netcdf(droughts_path)

# GENERA Base de Huracanes, con resolución completa

In [ ]:
## FIXED Parameters
sshws_min_wind = {
    # Saffir-Simpson Hurricane Wind Scale
    # Measured in knots
    # https://www.nhc.noaa.gov/aboutsshws.php
    5: 137,
    4: 113,
    3: 96,
    2: 83,
    1: 64,
}

agency_measurements = {
    "USA": 1, # 1-m measurement
    "TOK": 3, # 3-m measurement
    "CMA": 2, # 2-m measurement
    "HKO": 10, # 10-m measurement
    "KMA": 10,
    "NEW": 3,
    "REU": 10,
    "BOM": 10,
    "NAD": 10,
    "WEL": 10,
    "DS8": 1,
    "TD6": 1,
    "TD5": 1,
    "NEU": 1,
    "MLC": 1,
}

conversion_factor_to_1m = {
    1: 1,
    2: (1.22/1.15+1.17/1.11)/2,
    3: (1.22/1.12+1.17/1.09)/2,
    10: (1.22/1.06+1.17/1.05)/2,
}

agency_1m_conversion_factor = {
    k:conversion_factor_to_1m[v] for k, v in agency_measurements.items()
}

## Functions
def convert_wind_to_1m(wind, agency):
    if agency in agency_1m_conversion_factor:
        return wind * agency_1m_conversion_factor[agency]
    return wind

def convert_wind_to_sshws(wind):
    for cat, min_wind in sshws_min_wind.items():
        if wind >= min_wind:
            return cat
    return 0



In [ ]:
import geopandas as gpd

gdf = gpd.read_file(r"D:\Datasets\International Best Track Archive for Climate Stewardship (IBTrACS)\IBTrACS.ALL.list.v04r01.lines.shp")#, "BASIN", "SUBBASIN", "NAME", "ISO_TIME", "LAT", "LON", "WMO_WIND", "WMO_PRES", "WMO_AGENCY", "TRACK_TYPE"])

# Fill interpolated xy values
gdf["WMO_WIND"] = gdf["WMO_WIND"].ffill()

# Data from 1950 onwards
gdf["year"] = gdf["ISO_TIME"].str.split("-").str[0].astype(int)
gdf = gdf[gdf["year"] >= 1970]

# Convert each Agency wind to 1m-MSW
wind_cols = [col for col in gdf.columns if "_WIND" in col and "WMO_WIND" not in col]
agencies = [col.replace("_WIND", "") for col in wind_cols]

for col in tqdm(wind_cols):
    agency = col.split("_")[0]
    gdf[col] = gdf[col].apply(lambda x: convert_wind_to_1m(x, agency))

gdf["wind_speed"] = gdf[wind_cols].max(axis=1)

# Convert wind to SSHWS category
gdf["category"] = gdf["wind_speed"].apply(convert_wind_to_sshws)

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

buffers = [0.1, 0.25, 0.50, 1.00]
years = gdf["year"].unique()

paths_by_year = {}
for year in tqdm(years, leave=False):
    paths_by_year[year] = []
    for buffer in buffers:
        ### Filter year and create raster map based on the buffered best-track 
        ###     of the hurricane
        print(buffer)
        gdf_year = gdf[gdf.year == year]
        gdf_year = gdf_year[["wind_speed", "geometry"]].fillna(0)

        # Apply buffer to center of the storm
        gdf_year["geometry"] = gdf_year.geometry.buffer(buffer)
        
        # Make the biggest shock at a certain location the one shown in the xr.dataset 
        gdf_year = gdf_year.sort_values("wind_speed", ascending=True) 
        
        raster = make_geocube(
            vector_data=gdf_year,
            like=population,
        )
        raster = raster.assign_coords({"year": year})

        for category in [3, 4, 5]:
            ### Once the raster wind_speed is created, create a new boolean raster 
            ###     where the winds are greater than the minimum for the category
            varname = f"category_{category}_b{int(buffer*100)}"
            # Keep only hurricanes of a certain category
            raster_b = xr.where(
                raster.rename({"wind_speed":varname})[varname] >= sshws_min_wind[category], 
                True, 
                False
            )
            # Transform wind_speed to boolean
            raster_path = rf"{DATA_PROC}\shocks_by_grid\hurricanes_{year}_{varname}.nc"
            raster_b.to_netcdf(raster_path, encoding={varname:{"zlib": True, "complevel": 7}})
            paths_by_year[year] += [raster_path]
            break
        # xr.concat(dss, dim="year")


In [ ]:
from dask.distributed import Client
client = Client(memory_limit='7GB')
client

In [ ]:
client

In [ ]:
xr.open_dataset(files_year[0])["category_3_b10"].encoding["chunksizes"]

In [ ]:
### Compile all the data into a single dataset
path = rf"{DATA_PROC}\shocks_by_grid"
files = os.listdir(path)
files = [f for f in files if "hurricanes_" in f and f.endswith(".nc")]

dss = []
for year in range(1970, 2021):
    
    files_year = [os.path.join(path, f) for f in files if f"{year}" in f]
    ds = xr.open_mfdataset(files_year,  parallel=True, chunks="auto")
    ds = ds.assign_coords({"year": year})
    dss += [ds]
    
ds = xr.concat(dss, dim="year")
ds.to_netcdf(r"D:\World Bank\CLIENT v2\Data\Data_out\IBTrACS_hurricanes_yearly.nc")

In [ ]:
ds.to_netcdf(r"D:\World Bank\CLIENT v2\Data\Data_out\IBTrACS_hurricanes_yearly.nc")

In [ ]:
# Visualizer
year = 2020
hurr_by_name = gdf[(gdf.NAME == "BELNA") & (gdf.year == year)]
fig, ax = plt.subplots()

hurr_by_name.plot(column="category", legend=True, ax=ax)

xmin, ymin, xmax, ymax = hurr_by_name.total_bounds
hurr_proc.sel(year=year, x=slice(xmin, xmax), y=slice(ymax, ymin))["category_1_b10"].plot(ax=ax, cmap="Greys", alpha=0.5)